*The model was constructed for OS event as an example*

1. Import necessary files 

In [ ]:
# import original data in batch 1
ori_data<-read.csv("/localhome/bs22tmhn/[ResearchProject]/Batch1/survData_batch1.csv",row.names = 1)
# modify the EnsemblID
colnames(ori_data)<-sub("\\..*", "",colnames(ori_data))

# Import the list of candidate genes in batch 1
candidate_genes<-read.csv("/localhome/bs22tmhn/[ResearchProject]/candidate_genes_OS.csv",row.names = 1)

2. Select top 25 candidate protein coding genes ranked by abs_beta_coxcont 

In [ ]:
top25_pcg <- subset(candidate_genes, gene_biotype_cox50 == "protein_coding")
top25_pcg <- top25_pcg[order(top25_pcg$abs_beta_coxcont, decreasing = TRUE), ]
top25_pcg <- top25_pcg$Variable[1:25]

top25_pcg_reads<-ori_data[!is.na(ori_data$status),colnames(ori_data) %in% top25_pcg]

3. Remove genes with high correlation with others in top 25 candidate protein coding genes

In [ ]:
# Calculate the correlation coefficient of each pair of genes in top25_pcg_reads
top25_pcg_reads_cor <- cor(top25_pcg_reads)      # a correlation matrix

# identify pairs of genes with high correlation (>0.75):
# retrieve the location indices of gene pairs with high correlation (>0.75) in the matrix
highcor<-which(abs(top25_pcg_reads_cor) > 0.75 & top25_pcg_reads_cor!=1 & upper.tri(top25_pcg_reads_cor), arr.ind = TRUE)   
# get EnsemblID (variables) from the correlation matrix
varnames <- colnames(top25_pcg_reads_cor)

# create a data frame of gene pairs with high correlation
result <- data.frame(var1 = varnames[highcor[,1]],
                     var2 = varnames[highcor[,2]])

Overall, there are 15 pairs of 10 genes with high correlation. Among these 10 genes, 8 will be removed and only ENSG00000148677 and ENSG00000008056 are kept, because they are the smallest number of genes not highly correlated with each other, and each of them is directly correlated with the 4 different remaining genes.

In [ ]:
#remove gene
removed<-unlist(result)[!(unlist(result)=="ENSG00000148677"|unlist(result)=="ENSG00000008056")]  # exclude ENSG00000148677 and ENSG00000008056 from the "removed" list containing genes to be removed
removed<-array(removed[!duplicated(removed)])  

top25_pcg_reads<-top25_pcg_reads[,!colnames(top25_pcg_reads) %in% removed]     #remove the 8 genes in the "removed" array, 17 genes remained to build the model

In [ ]:
# create a data frame containing the selected genes and essential clinical data
surv<-cbind(top25_pcg_reads, ori_data[!is.na(ori_data$status),c("OS_time","status","NHLDeath")])
surv$status<-ifelse(surv$NHLDeath==1 & !is.na(surv$NHLDeath),1,0)
surv<-surv(,-ncol(surv))   # remove the last column (NHLDeath) of surv

4. Test for multicollinearity in the remaining genes

In [ ]:
pretend.lm<-lm(OS_time~ .,data=surv[,1:18])  # surv[,1:18] contains 17 genes and OS_time
car::vif(pretend.lm)

Based on the output of *car::vif(pretend.lm)*, there are many genes having VIF > 2.5, suggesting the presence of multicollinearity in this set of genes. Thus, Ridge penalised Cox regression is used to mitigate this problem in model construction.

5. Model construction using L2 (Ridge) Cox regression

In [ ]:
library(penalized)
library(survival)

In [ ]:
# perform 10 fold cross validation to find the optimised lambda
set.seed(123)
opt2<-optL2(Surv(OS_time, status) ~ . , data=surv,fold=10)

# fit Cox PH model to the data with optimised L2 lambda=0.19
fit2<-penalized(Surv(OS_time, status) ~ . , data=surv,lambda2=0.19)

# store the penalised coefficient estimates 
coef2<-coefficients(fit2)

6. Using the coefficients, compute the signature score for each sample and split the samples into high- and low-score groups at the median score

In [ ]:
# calculate the signature by multiplying the gene expression values to their corresponding coefficients in each sample
surv$score2<-as.matrix(surv[,c(1:17)]) %*% as.matrix(coef2)
surv$score2<-surv$score2[,1]

# split the data set into 2 groups using median as threshold
med2<-median(surv$score2)
# Group 1 below median, Group 2 above median
surv$median2 <- ifelse(surv$score2 <= med2, 1, 2)

# run Cox PH to estimate HR of the high- and low-score groups split at median
L2_bin<-coxph(Surv(OS_time,status)~median2,data=surv)

7. Estimate the average 10-year OS rate of two groups of patients (in batch 1) classified by the model

In [ ]:
# fit the model
L2<-coxph(Surv(OS_time,status)~score2,data=surv)

# Predict the survival probability of each sample at 10 years
surv_test<-data.frame(score2=surv$score2)
surv_test$status<-1
surv_test$OS_time<-10
pred <- predict(L2, newdata = surv_test, type = "expected", se.fit = T)    # Predict the expected number of events at 10 years
# Calculate the survival probability
sprob <- exp(-pred$fit)
# append the survival probability to "surv" dataframe
surv$prob10<-sprob

# calculate the average 10-year OS rate of high-score group
mean(surv$prob10[surv$median2 == 2])
# calculate the average 10-year OS rate of low-score group
mean(surv$prob10[surv$median2 == 1])